<h1><center>Laboratorio 9: Optimización de modelos 💯</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Ignacio Meza, Gabriel Iturra
- Auxiliar: Sebastián Tinoco
- Ayudante: Arturo Lazcano, Angelo Muñoz

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Maximiliano Westerhout Aliste
- Nombre de alumno 2: Álvaro Gallardo Alvarado


## Temas a tratar

- Predicción de demanda usando `xgboost`
- Búsqueda del modelo óptimo de clasificación usando `optuna`
- Uso de pipelines.

## Reglas:

- **Grupos de 2 personas**
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias.
- Pueden usar cualquer material del curso que estimen conveniente.

### Objetivos principales del laboratorio

- Optimizar modelos usando `optuna`
- Recurrir a técnicas de *prunning*
- Forzar el aprendizaje de relaciones entre variables mediante *constraints*
- Fijar un pipeline con un modelo base que luego se irá optimizando.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre DataFrames.

### **Link de repositorio de GitHub:** `https://github.com/MaxWesterhout/Lab-9-Progra-cientifica.git`

# Importamos librerias útiles

In [1]:
!pip install -qq xgboost optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.6/409.6 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.8/226.8 kB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 5.8 MB/s eta 0:00:00


# 1. El emprendimiento de Fiu

Tras liderar de manera exitosa la implementación de un proyecto de ciencia de datos para caracterizar los datos generados en Santiago 2023, el misterioso corpóreo **Fiu** se anima y decide levantar su propio negocio de consultoría en machine learning. Tras varias e intensas negociaciones, Fiu logra encontrar su *primera chamba*: predecir la demanda (cantidad de venta) de una famosa productora de bebidas de calibre mundial. Como usted tuvo un rendimiento sobresaliente en el proyecto de caracterización de datos, Fiu lo contrata como *data scientist* de su emprendimiento.

Para este laboratorio deben trabajar con los datos `sales.csv` subidos a u-cursos, el cual contiene una muestra de ventas de la empresa para diferentes productos en un determinado tiempo.

Para comenzar, cargue el dataset señalado y visualice a través de un `.head` los atributos que posee el dataset.

<i><p align="center">Fiu siendo felicitado por su excelente desempeño en el proyecto de caracterización de datos</p></i>
<p align="center">
  <img src="https://media-front.elmostrador.cl/2023/09/A_UNO_1506411_2440e.jpg">
</p>

In [3]:
import pandas as pd
import numpy as np
from datetime import datetime
import pickle

df = pd.read_csv('sales.csv')
df['date'] = pd.to_datetime(df['date'])

df.head()

,id,date,city,lat,long,pop,shop,brand,container,capacity,price,quantity
0,0,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,glass,500ml,0.96,13280
1,1,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,plastic,1.5lt,2.86,6727
2,2,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,can,330ml,0.87,9848
3,3,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,adult-cola,glass,500ml,1.00,20050
4,4,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,adult-cola,can,330ml,0.39,25696


In [4]:
for i in df.columns.to_list():
  print('Unique data',i,':', df[i].nunique())

Unique data id : 7456
Unique data date : 84
Unique data city : 5
Unique data lat : 6
Unique data long : 6
Unique data pop : 35
Unique data shop : 6
Unique data brand : 5
Unique data container : 3
Unique data capacity : 3
Unique data price : 402
Unique data quantity : 6906


Se considerara la variable POP como categorica dado que implica un codigo de area, independiente que la data que contiene sea numerica, siguen siendo categorias. Lo mismo con latitud y longitud, que consideran ubicaciones no numeros directamente.

Por lo tanto, la unica variable numerica es price, el resto corresponde a categoricas.

## 1.1 Generando un Baseline (0.5 puntos)

<p align="center">
  <img src="https://media.tenor.com/O-lan6TkadUAAAAC/what-i-wnna-do-after-a-baseline.gif">
</p>

Antes de entrenar un algoritmo, usted recuerda los apuntes de su magíster en ciencia de datos y recuerda que debe seguir una serie de *buenas prácticas* para entrenar correcta y debidamente su modelo. Después de un par de vueltas, llega a las siguientes tareas:

1. Separe los datos en conjuntos de train (70%), validation (20%) y test (10%). Fije una semilla para controlar la aleatoriedad.
2. Implemente un `FunctionTransformer` para extraer el día, mes y año de la variable `date`. Guarde estas variables en el formato categorical de pandas.
3. Implemente un `ColumnTransformer` para procesar de manera adecuada los datos numéricos y categóricos. Use `OneHotEncoder` para las variables categóricas.
4. Guarde los pasos anteriores en un `Pipeline`, dejando como último paso el regresor `DummyRegressor` para generar predicciones en base a promedios.
5. Entrene el pipeline anterior y reporte la métrica `mean_absolute_error` sobre los datos de validación. ¿Cómo se interpreta esta métrica para el contexto del negocio?
6. Finalmente, vuelva a entrenar el `Pipeline` pero esta vez usando `XGBRegressor` como modelo **utilizando los parámetros por default**. ¿Cómo cambia el MAE al implementar este algoritmo? ¿Es mejor o peor que el `DummyRegressor`?
7. Guarde ambos modelos en un archivo .pkl (uno cada uno)

In [7]:
from sklearn.preprocessing import StandardScaler
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import OneHotEncoder

class one_hot(BaseEstimator, TransformerMixin):
  def __init__(self, min_frequency_param = None):
    #min_frequency_param=encoder_params['min_frequency']
    self.min_frequency_param = min_frequency_param

  def fit(self,x,y=None):
    return self

  def transform(self,x):
    #Onehot variables categoricas
    minima_frequency = self.min_frequency_param
    enc = OneHotEncoder(min_frequency=minima_frequency)
    enc_df = pd.DataFrame(enc.fit_transform(x).toarray())

    x = enc_df
    x.columns = x.columns.astype(str)
    return x

  def get_feature_names_out(self):
    pass

def separacion(df):
    # Pasamos a datetime
    df['date'] = pd.to_datetime(df['date'])
    # Utilizamos las variables de datetime para separar las columnas
    df['dia'] = df['date'].dt.day.astype('category')
    df['mes'] = df['date'].dt.month.astype('category')
    df['año'] = df['date'].dt.year.astype('category')
    return df.drop('date', axis=1)


In [8]:
from  sklearn.model_selection import train_test_split
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.dummy import DummyRegressor

#La variable ID no se considera dado que solo es un indexador de los datos, no informacion del problema
X_labels = ['date',	'city',	'lat',	'long',	'pop',	'shop',	'brand',	'container',	'capacity',	'price']
Y_labels = ['quantity']

#Variables numericas
numeric_var = ['price','pop','lat','long',]
#Variables categoricas (POP ES CATEGORICA?)
categ_var = ['city','shop','brand','container','capacity', 'año','mes','dia']


# Inserte su código acá
test_size_df = round(0.1*len(df))
val_size_df = round(0.2*len(df))

#Random state 42 = semilla
X_train, X_test_final = train_test_split(df, test_size= test_size_df, random_state=42)
X_train_final, X_val_final = train_test_split(X_train, test_size= val_size_df, random_state=42)

Y_train_final = X_train_final.reset_index(drop=True)[Y_labels]
Y_val_final = X_val_final.reset_index(drop=True)[Y_labels]
Y_test_final = X_test_final.reset_index(drop=True)[Y_labels]

X_train_final = X_train_final.reset_index(drop=True)[X_labels]
X_val_final = X_val_final.reset_index(drop=True)[X_labels]
X_test_final = X_test_final.reset_index(drop=True)[X_labels]

#X_train_final, X_temp, Y_train_final, y_temp = train_test_split(df.drop('quantity', axis=1), df['quantity'], test_size=0.3, random_state=42)
#X_val_final, X_test, Y_val_final, y_test = train_test_split(X_temp, y_temp, test_size=0.33, random_state=42)


#Implementacion FunctionTransformer conjunto train
separacion_func = FunctionTransformer(separacion, validate=False)

#Implementacion ColumnTransformer
ct = ColumnTransformer(transformers = [
    ('Procesamiento_numericas',StandardScaler(),numeric_var),
    ('Procesamiento_categoricas', one_hot(),categ_var)],remainder="passthrough",verbose_feature_names_out=False)

ct.set_output(transform='pandas')

pipeline = Pipeline([
    ('function_transform',separacion_func),
    ('transformacion',ct),
    ('regresor', DummyRegressor(strategy = 'mean'))
])

In [9]:
import joblib
#Entrenamos el pipeline
primer_pipeline = pipeline.fit(X_train_final,Y_train_final)

joblib.dump(primer_pipeline, 'pipeline_model.pkl')

['pipeline_model.pkl']

In [10]:
from sklearn.metrics import mean_absolute_error
# Predecir con el pipeline
predictions = primer_pipeline.predict(X_val_final)

# Evaluar el rendimiento del modelo utilizando mean_absolute_error
mae = mean_absolute_error(Y_val_final, predictions)
print(f'Mean Absolute Error DummyRegressor: {mae}')

Mean Absolute Error DummyRegressor: 13546.494391911923


In [11]:
import xgboost as xgb

modelo = xgb.XGBRegressor()

#Nuevo pipeline con XGBRegressor
seg_pipeline = Pipeline([
    ('function_transform',separacion_func),
    ('transformacion',ct),
    ('regresor', modelo)
])

#Entrenamos el pipeline
segundo_pipeline = seg_pipeline.fit(X_train_final,Y_train_final)

joblib.dump(segundo_pipeline, 'pipeline_model_2.pkl')

['pipeline_model_2.pkl']

In [12]:
# Predecir con el pipeline
predictions = segundo_pipeline.predict(X_val_final)

# Evaluar el rendimiento del modelo utilizando mean_absolute_error
mae = mean_absolute_error(Y_val_final, predictions)
print(f'Mean Absolute Error XGBRegressor: {mae}')

Mean Absolute Error XGBRegressor: 2390.122370125702


¿Cómo se interpreta esta métrica para el contexto del negocio?
Esta metrica corresponde al Error abosoluto medio (Mean Absolute Error), siendo la diferencia absoluta del promedio de los datos, siendo los datos en este caso las etiquetas (cantidad/quantity) reales en comparacion a la prediccion. Por lo tanto, este valor implica que existe un margen de error de 13546.49 (DumyRegressor) y 2762.26 (XGBRegressor) referido a las predicciones de cantidad, en otras palabras, el valor real para el primer clasificador estara entre [-13546.49 + predicion, prediccion + 13546.49] y para el segundo sera de [-2762.26 + prediccion, prediccion + 2762.26]. Por lo tanto, un menor MAE implica una mejor precision al momento de predecir la cantidad.


¿Cómo cambia el MAE al implementar este algoritmo? ¿Es mejor o peor que el DummyRegressor?

El MAE se redujo en una gran cantidad, esto implica que el clasificador tiende a tener un menor error promedio en su clasificacion lo que se traduce en una mejor precision. Por ello corresponde a un mejor clasificador en terminos de poseer un menor error asociado a sus predicciones.

## 1.2 Forzando relaciones entre parámetros con XGBoost (1.0 puntos)

<p align="center">
  <img src="https://64.media.tumblr.com/14cc45f9610a6ee341a45fd0d68f4dde/20d11b36022bca7b-bf/s640x960/67ab1db12ff73a530f649ac455c000945d99c0d6.gif">
</p>

Un colega aficionado a la economía le *sopla* que la demanda guarda una relación inversa con el precio del producto. Motivado para impresionar al querido corpóreo, se propone hacer uso de esta información para mejorar su modelo.

Vuelva a entrenar el `Pipeline`, pero esta vez forzando una relación monótona negativa entre el precio y la cantidad. Luego, vuelva a reportar el `MAE` sobre el conjunto de validación. ¿Cómo cambia el error al incluir esta relación? ¿Tenía razón su amigo?

Nuevamente, guarde su modelo en un archivo .pkl

Nota: Para realizar esta parte, debe apoyarse en la siguiente <a href = https://xgboost.readthedocs.io/en/stable/tutorials/monotonic.html>documentación</a>.

Hint: Para implementar el constraint, se le sugiere hacerlo especificando el nombre de la variable. De ser así, probablemente le sea útil **mantener el formato de pandas** antes del step de entrenamiento.

In [14]:
# XGB CONSTRAINT
params_constrained = {'price': -1}

regressor = xgb.XGBRegressor(monotone_constraints=params_constrained)

#regressor.set_params(monotone_constraints=params_constrained)

pipeline = Pipeline([
    ('function_transform', separacion_func),
    ('transformacion', ct),
    ('regresor_constraint', regressor)  # Pasa el regresor creado con los parámetros de restricción
])


Tercer_pipeline = pipeline.fit(X_train_final, Y_train_final)
val_predictions_xgb_inv = pipeline.predict(X_val_final)
mae_xgb_inv = mean_absolute_error(Y_val_final, val_predictions_xgb_inv)

print(f'MAE con XGBoost: {mae_xgb_inv}')

MAE con XGBoost: 2529.2283472973254


¿Cómo cambia el error al incluir esta relación? ¿Tenía razón su amigo?

Para este caso, el MAE aumento lo cual implica un peor resultado. Esto significa que probablemente la variable price no posee una relacion monotona negativa con la variable quantity y por ello forzar dicha relacion genera un peor resultado, por lo tanto no tenia razon mi amigo.

Cabe destacar que se hizo un analisis de la data referido a esta hipotesis viendo lo siguiente:

In [27]:
df_2 = df.copy()
df_2 = df_2.sort_values(by='quantity', ascending=False)

df_2_1 = df_2[:3600]
df_2_2 = df_2[3600:7200]

new_df = {'price_primera_mitad':df_2_1['price'],'quantity_primera_mitad':df_2_1['quantity'],'price_segunda_mitad':df_2_2['price'],'quantity_segunda_mitad':df_2_2['quantity']}

df_3 = pd.DataFrame(new_df)
df_3.describe()

,price_primera_mitad,quantity_primera_mitad,price_segunda_mitad,quantity_segunda_mitad
count,3600.000000,3600.000000,3600.000000,3600.000000
mean,0.802828,43034.822222,1.494722,17402.856944
std,0.474408,15971.740416,0.833829,4843.202315
min,0.110000,25964.000000,0.200000,8187.000000
25%,0.490000,31429.250000,0.837500,13486.500000
50%,0.660000,38294.500000,1.230000,17492.000000
75%,0.980000,50026.000000,2.080000,21438.250000
max,3.210000,145287.000000,4.320000,25956.000000


Dicho analisis expone que si ordenamos los datos de forma descendente (desde el menor al mayor) de price, si separamos el dataframe en dos obtenemos dos nuevos conjuntos con sus propios datos de price y quantity. Dicha informacion arroja que para la primera mitad que contiene los precios mas bajos poseen un promedio de quantity mucho mayor que el segundo conjunto, el cual posee prices mucho mayores pero a la vez cantidades menores. Debido a este resultado pareciera existir una relacion inversa referida a estas variables, lo cual se asocai con la monotonidad negativa.

Debido a esto, encontramos extraño de que no disminuya el error dado que el supuesto pareciera ser correcto. Aqui nosotros suponemos que el error esta asociado a que la variable cuantity es la etiqueta del problema, por ello al aplicar la restriccion de monotocidad pareciera no afectar a dicha variable dado que es externa al conjunto de entrenamiento. Buscamos formas de solucionar esto pero pareciera que la unica forma es entrenar dos modelos, uno que contenga el conjunto de variables y otra que no, pero eso seria una especie de trampa dado que se entrenaria el modelo con informacion de las etiquetas.

Por lo tanto, como grupo pensamos que el supuesto es correcto pero a priori no sabemos como agregar dicha informacion con un constraint correcto.

Pd: Existe otra explicacion que puede estar asociada a la forma que se distribuyen lo datos, dado que independiente de que los promedios parecieran ser inversos no sabemos a priori como distribuyen los datos, puede que sea por los outlayers.

## 1.3 Optimización de Hiperparámetros con Optuna (2.0 puntos)

<p align="center">
  <img src="https://media.tenor.com/fmNdyGN4z5kAAAAi/hacking-lucy.gif">
</p>

Luego de presentarle sus resultados, Fiu le pregunta si es posible mejorar *aun más* su modelo. En particular, le comenta de la optimización de hiperparámetros con metodologías bayesianas a través del paquete `optuna`. Como usted es un aficionado al entrenamiento de modelos de ML, se propone implementar la descabellada idea de su jefe.

A partir de la mejor configuración obtenida en la sección anterior, utilice `optuna` para optimizar sus hiperparámetros. En particular, se le pide:

- Fijar una semilla en las instancias necesarias para garantizar la reproducibilidad de resultados
- Utilice `TPESampler` como método de muestreo
- De `XGBRegressor`, optimice los siguientes hiperparámetros:
    - `learning_rate` buscando valores flotantes en el rango (0.001, 0.1)
    - `n_estimators` buscando valores enteros en el rango (50, 1000)
    - `max_depth` buscando valores enteros en el rango (3, 10)
    - `max_leaves` buscando valores enteros en el rango (0, 100)
    - `min_child_weight` buscando valores enteros en el rango (1, 5)
    - `reg_alpha` buscando valores flotantes en el rango (0, 1)
    - `reg_lambda` buscando valores flotantes en el rango (0, 1)
- De `OneHotEncoder`, optimice el hiperparámetro `min_frequency` buscando el mejor valor flotante en el rango (0.0, 1.0)
- Explique cada hiperparámetro y su rol en el modelo. ¿Hacen sentido los rangos de optimización indicados?
- Fije el tiempo de entrenamiento a 5 minutos
- Reportar el número de *trials*, el `MAE` y los mejores hiperparámetros encontrados. ¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto?
- Guardar su modelo en un archivo .pkl

¿Hacen sentido los rangos de optimización indicados?

Tenemos la siguiente serie de parametros:

1)Learnging rate: Este parametro corresponde a la taza de aprendizaje del modelo y el limite escogido es entre (0.001 y 0.1). Este valor controla el tamaño del paso con el que el algoritmo actualiza sus pesos y por lo general debe poseer valores bajos para garantizar que la solucion llegue a su optimo de manera correcta (slow and smoothly). Si los pasos fueran muy grandes, el valor no convergeria correctamente y quedaria en un loop constante, del mismo modo si es muy pequeño tardaria mucho en converger. El valor estandar para este parametro del XGBOOST es de 0.3. Una forma de imaginar por que estos limites son correctos, es imaginar el descenso del gradiente, en donde tazas de aprendizaje muy altas provocan que nunca se llegue al minimo abosuluto que corresponde al de optimmizacion. Por lo tanto tiene sentido esos rangos.

2)n_estimators: Corresponde al numero de arboles presentes en el algoritmo, por lo general numeros altos generan mejores resultados pero un exceso de este parametro genera overfitting. Por lo general los rangos que se acostumbran utilizar corresponden entre 100 y 1000, seindo los limites de este caso (50, 1000). Por lo tanto, estos limites tienen sentido bajo la logica general.

3)max_depth: Corresponde a la profundidad de los arboles, que se asocia con el n_estimators (numero de arboles). Al aumentar el numero de este parametro se generan modelos con mas produndidad o capas, esto genera modelos mas complejos pero un numero excesivo de capas tambien genera overfitting. El valor estandar para este parametro es de 3, seindo el rango escogido para este caso (3,10), lo cual tiene sentido. Aun asi, un modelo con profundidad de 10 corresponde a un modelo complejo por lo cual se debe revisar si existe overffiting.

4)max_leaves: Numero de hojas o nodos correspondiente a los arboles del modelo, similar al caso anterior un numero mayor se traduce en una moyar complejidad del modelo con un mayor numero de pesos, pero nuevamente un numero grande puede generar overfitting. El valore estandar para este parametro corresponde a 0, dado que no se establece un limite, para este caso se utilizan los limites (0,100), lo cual tiene sentido bajo la logica debido a que si no establecemos un limite el modelo puede tardar mucho en entrenar, siendo 100 un numero razonable (puede ser mas).

5)min_child_weight: Este corresponde al minimo numero de instancias presentes en cada nodo/hijo del arbol, mientras mayor sea el valor mas veces se revisaran cada nodo del arbol, siendo el valor estandar 1. Para este caso el rango es de (1,5) lo cual tiene sentido dado que si el rango fuera mayor generaria que el modelo tardaria mucho mas.

6)reg_alpha: Este parametro corresponde a un valor asociado a la "Lasso regularization", dicha tecnica permite evitar el overfitting teniendo valores entre 0 a infinito. Para este caso, se escogio los rangos de (0,1) lo cual no esta mal, si se observa un valor muy overfitieado es recomendable aumentar dicho rango.

7)reg_lambda: Corresponde al coeficiente asociado a la "ridge regression", que aplica “squared magnitude” a la funcion loss. Esta tecnica es similar a la anterior generan una penalizacion que provoca que se evite el overffiting dado que operan en la funcion loss. El rango para este caso es de (0,1) siendo bueno, si se posee mucho overfitting se recomienda aumentar el rango.

8)min_frequency: Corresponde a la frecuencia minima del OneHotEncoder, el valor establece el tipo de codificacion que se genera, si es int las categorías con una cardinalidad menor se considerarán poco frecuentes y si es float,
las categorías con una cardinalidad menor que min_frequency * n_samples (lenght(columna)) se considerarán poco frecuentes. El rango utilizado para este caso es de (0.0, 1.0), por lo cual siempre se estaran tomando valores tipo float que provocaran cambios en el numero de columnas asociadas al onehotencoder. Esto ultimo tiene todo el sentido del mundo dado que si se quiere mejorar u optimizar el resultado de este clasificador, se deben alterar las codificaciones de las variables categoricas, en donde al establecer este rango de 0 a 1 por cada iteracion se presentaran distintas codificaciones y en consecuencia se obtendran distintos resultados. De hecho, si se imprime el numero de columnas se observa que varian dependiendo de la minima frequency presente.

Por lo tanto, respondiendo a la pregunta, si tienen sentido los rangos escogidos.

In [30]:
# Inserte su código acá
import optuna
from sklearn.impute import SimpleImputer
import time

#Se fija la semilla
seed = 10
import numpy as np
np.random.seed(seed)

#Parametros
parametros = {
    'learning_rate': (0.001, 0.1),
    'n_estimators': (50, 1000),
    'max_depth': (3, 10),
    'max_leaves': (0, 100),
    'min_child_weight': (1, 5),
    'reg_alpha': (0, 1),
    'reg_lambda': (0, 1),}

encoder_parametros = {
    'min_frequency': (0.0, 1.0)}

In [31]:
#Sintaxis optuna
def objective(trial):
  # Definir el espacio de búsqueda para XGBRegressor
    xgb_params = {
        'learning_rate': trial.suggest_float('learning_rate', *parametros['learning_rate']),
        'n_estimators': trial.suggest_int('n_estimators', *parametros['n_estimators']),
        'max_depth': trial.suggest_int('max_depth', *parametros['max_depth']),
        'max_leaves': trial.suggest_int('max_leaves', *parametros['max_leaves']),
        'min_child_weight': trial.suggest_int('min_child_weight', *parametros['min_child_weight']),
        'reg_alpha': trial.suggest_float('reg_alpha', *parametros['reg_alpha']),
        'reg_lambda': trial.suggest_float('reg_lambda', *parametros['reg_lambda']),
    }

    # Definir el espacio de búsqueda para OneHotEncoder
    encoder_params = {
        'min_frequency': trial.suggest_float('min_frequency', *encoder_parametros['min_frequency'])
    }

    Nuevo_one_hot_parametro = one_hot(min_frequency_param=encoder_params['min_frequency'])
    separacion_func = FunctionTransformer(separacion, validate=False)

    #Implementacion ColumnTransformer
    ct = ColumnTransformer(transformers = [
    ('Procesamiento_numericas',StandardScaler(),numeric_var),
    ('Procesamiento_categoricas', Nuevo_one_hot_parametro,categ_var)],remainder="passthrough",verbose_feature_names_out=False)

    ct.set_output(transform='pandas')

    #Generacion pipeline
    pipeline = Pipeline([
    ('function_transform',separacion_func),
    ('transformacion',ct),
    ('regresor', xgb.XGBRegressor(**xgb_params))
    ])

    start_time = time.time()

    # Entrenar el modelo
    pipeline.fit(X_train_final, Y_train_final)

    # Realizar predicciones en el conjunto de validación
    predictions_xgb = pipeline.predict(X_train_final)

    # Calcular el MAE
    mae_xgb = mean_absolute_error(Y_train_final, predictions_xgb)

    print(f'MAE: {mae_xgb}')

    return mae_xgb


# Configurar optuna
optuna.logging.set_verbosity(optuna.logging.WARNING)
sampler = optuna.samplers.TPESampler(seed=seed)
study_opt = optuna.create_study(direction='minimize', sampler=sampler)

# Ejecutar la optimización
study_opt.optimize(objective, timeout=300, show_progress_bar=True)

# Obtener los mejores hiperparámetros
best_params = study_opt.best_params
best_mae = study_opt.best_value
num_trials = len(study_opt.trials)

print(f'Número de trials: {num_trials}')
print(f'MAE con los mejores hiperparámetros: {best_mae}')
print(f'Mejores hiperparámetros: {best_params}')

   0%|          | 00:00/05:00

MAE: 10026.092779425237
MAE: 10080.961297138461
MAE: 10024.734471739377
MAE: 9812.802996151382
MAE: 10032.717595208762
MAE: 10024.891339973236
MAE: 10089.2441930177
MAE: 10024.75559572745
MAE: 10252.851591315984
MAE: 10018.160810036046
MAE: 5768.080133558347
MAE: 3708.533172812081
MAE: 3984.9681306434422
MAE: 908.9350286082853
MAE: 1265.3192152003946
MAE: 1372.2197414850637
MAE: 1332.3021884617804
MAE: 7199.349217786348
MAE: 7280.207288558969
MAE: 1068.8969962797441
MAE: 6805.616100866051
MAE: 867.2560050454169
MAE: 3277.289697153355
MAE: 833.1513495049638
MAE: 5418.4033749973805
MAE: 826.3086531128
MAE: 1020.2049559845187
MAE: 4142.808312700589
MAE: 6810.86558414305
MAE: 2802.18996643312
MAE: 801.2555454398142
MAE: 843.6299288432291
MAE: 3121.1144941877783
MAE: 4430.90615396231
MAE: 447.98300792477005
MAE: 7198.671915697901
MAE: 2417.581092625428
MAE: 530.4190047220792
MAE: 3236.844608024017
MAE: 986.8987966238893
MAE: 6656.554322996503
MAE: 340.7136809595199
MAE: 893.2874045985072
MA

In [33]:
# Obtener el mejor modelo con los mejores hiperparámetros
xgb_params_best = {
    'learning_rate': best_params['learning_rate'],
    'n_estimators': best_params['n_estimators'],
    'max_depth': best_params['max_depth'],
    'max_leaves': best_params['max_leaves'],
    'min_child_weight': best_params['min_child_weight'],
    'reg_alpha': best_params['reg_alpha'],
    'reg_lambda': best_params['reg_lambda'],
}

#Estandar
Nuevo_one_hot_parametro = one_hot(min_frequency_param=best_params['min_frequency'])
separacion_func = FunctionTransformer(separacion, validate=False)

#Implementacion ColumnTransformer
ct = ColumnTransformer(transformers = [
('Procesamiento_numericas',StandardScaler(),numeric_var),
('Procesamiento_categoricas', Nuevo_one_hot_parametro,categ_var)],remainder="passthrough",verbose_feature_names_out=False)

ct.set_output(transform='pandas')

#Generacion pipeline
best_model = Pipeline([
('function_transform',separacion_func),
('transformacion',ct),
('regresor', xgb.XGBRegressor(**xgb_params_best))
])


# Entrenar el mejor modelo con todos los datos
best_model.fit(X_train_final, Y_train_final)
val_predictions_xgb_opt = best_model.predict(X_val_final)
mae_xgb_opt = mean_absolute_error(Y_val_final, val_predictions_xgb_opt)
print(f'MAE con XGBoost Optimizado: {mae_xgb_opt}')
print(f'Número de trials: {num_trials}')
#Print parametros
print('')
print('Mejores hiperparámetros:')
for key, value in best_params.items():
    print(f'{key}: {value}')

# Guardar modelo
joblib.dump(best_model, 'best_model_optimized_primero.pkl')

MAE con XGBoost Optimizado: 2094.7839817223653
Número de trials: 132

Mejores hiperparámetros:
learning_rate: 0.07722139907687305
n_estimators: 978
max_depth: 8
max_leaves: 99
min_child_weight: 2
reg_alpha: 0.2385838098063926
reg_lambda: 0.23342325029414504
min_frequency: 0.029518033187261532


['best_model_optimized_primero.pkl']

¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto?

Se obtuvieron mejores resultados, teniendo un MAE de 2094.78 que se traduce como una mejor precision al momento de predecir las cantidades. Este resultado se debe a la optimizacion anterior, dado que al utilzar los mejores parametros se garantiza obtener el mejor valor para la metrica objetivo siendo este caso el MAE.

Este resultado se debe principalmente a la utilizacion del parametro min_frequency, en donde al alterar la codificación se generan nuevos resultados, que en conjunto con los mejores parametros del clasificador se logra un mejor desempeño.

Nuevo resultado = disminuyo en 296 el MAE

## 1.4 Optimización de Hiperparámetros con Optuna y Prunners (1.7)

<p align="center">
  <img src="https://i.pinimg.com/originals/90/16/f9/9016f919c2259f3d0e8fe465049638a7.gif">
</p>

Después de optimizar el rendimiento de su modelo varias veces, Fiu le pregunta si no es posible optimizar el entrenamiento del modelo en sí mismo. Después de leer un par de post de personas de dudosa reputación en la *deepweb*, usted llega a la conclusión que puede cumplir este objetivo mediante la implementación de **Prunning**.

Vuelva a optimizar los mismos hiperparámetros que la sección pasada, pero esta vez utilizando **Prunning** en la optimización. En particular, usted debe:

- Responder: ¿Qué es prunning? ¿De qué forma debería impactar en el entrenamiento?
- Utilizar `optuna.integration.XGBoostPruningCallback` como método de **Prunning**
- Fijar nuevamente el tiempo de entrenamiento a 5 minutos
- Reportar el número de *trials*, el `MAE` y los mejores hiperparámetros encontrados. ¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto?
- Guardar su modelo en un archivo .pkl

Nota: Si quieren silenciar los prints obtenidos en el prunning, pueden hacerlo mediante el siguiente comando:

```
optuna.logging.set_verbosity(optuna.logging.WARNING)
```

De implementar la opción anterior, pueden especificar `show_progress_bar = True` en el método `optimize` para *más sabor*.

Hint: Si quieren especificar parámetros del método .fit() del modelo a través del pipeline, pueden hacerlo por medio de la siguiente sintaxis: `pipeline.fit(stepmodelo__parametro = valor)`

Hint2: Este <a href = https://stackoverflow.com/questions/40329576/sklearn-pass-fit-parameters-to-xgboost-in-pipeline>enlace</a> les puede ser de ayuda en su implementación

In [35]:
# Inserte su código acá
from optuna.integration import XGBoostPruningCallback
def objective(trial):
  # Definir el espacio de búsqueda para XGBRegressor
    xgb_params = {
        'learning_rate': trial.suggest_float('learning_rate', *parametros['learning_rate']),
        'n_estimators': trial.suggest_int('n_estimators', *parametros['n_estimators']),
        'max_depth': trial.suggest_int('max_depth', *parametros['max_depth']),
        'max_leaves': trial.suggest_int('max_leaves', *parametros['max_leaves']),
        'min_child_weight': trial.suggest_int('min_child_weight', *parametros['min_child_weight']),
        'reg_alpha': trial.suggest_float('reg_alpha', *parametros['reg_alpha']),
        'reg_lambda': trial.suggest_float('reg_lambda', *parametros['reg_lambda']),
    }

    # Definir el espacio de búsqueda para OneHotEncoder
    encoder_params = {
        'min_frequency': trial.suggest_float('min_frequency', *encoder_parametros['min_frequency'])
    }

    Nuevo_one_hot_parametro = one_hot(min_frequency_param=encoder_params['min_frequency'])
    separacion_func = FunctionTransformer(separacion, validate=False)

    #Implementacion ColumnTransformer
    ct = ColumnTransformer(transformers = [
    ('Procesamiento_numericas',StandardScaler(),numeric_var),
    ('Procesamiento_categoricas', Nuevo_one_hot_parametro,categ_var)],remainder="passthrough",verbose_feature_names_out=False)

    ct.set_output(transform='pandas')

    #Generacion pipeline
    pipeline = Pipeline([
    ('function_transform',separacion_func),
    ('transformacion',ct),
    ('regresor', xgb.XGBRegressor(**xgb_params))
    ])

    start_time = time.time()


    # Entrenar el modelo
    pipeline.fit(X_train_final, Y_train_final)

    # Realizar predicciones en el conjunto de validación
    predictions_xgb = pipeline.predict(X_train_final)

    pruning_callback = XGBoostPruningCallback(trial, 'validation-mae')
    # Calcular el MAE
    mae_xgb = mean_absolute_error(Y_train_final, predictions_xgb)

    print(f'MAE: {mae_xgb}')

    return mae_xgb

In [36]:
sampler = optuna.samplers.TPESampler(seed=seed)
study = optuna.create_study(direction='minimize', sampler=sampler)
# Configurar el tiempo límite de la optimización a 5 minutos
study.optimize(objective, timeout=300, show_progress_bar = True)

# Obtener los mejores hiperparámetros y su rendimiento
best_params = study.best_params
best_mae = study.best_value
num_trials = len(study.trials)

print(f"Número de trials: {num_trials}")
print(f"MAE: {best_mae}")
print(f"Mejores hiperparámetros: {best_params}")

   0%|          | 00:00/05:00

MAE: 10026.092779425237
MAE: 10080.961297138461
MAE: 10024.734471739377
MAE: 9812.802996151382
MAE: 10032.717595208762
MAE: 10024.891339973236
MAE: 10089.2441930177
MAE: 10024.75559572745
MAE: 10252.851591315984
MAE: 10018.160810036046
MAE: 5768.080133558347
MAE: 3708.533172812081
MAE: 3984.9681306434422
MAE: 908.9350286082853
MAE: 1265.3192152003946
MAE: 1372.2197414850637
MAE: 1332.3021884617804
MAE: 7199.349217786348
MAE: 7280.207288558969
MAE: 1068.8969962797441
MAE: 6805.616100866051
MAE: 867.2560050454169
MAE: 3277.289697153355
MAE: 833.1513495049638
MAE: 5418.4033749973805
MAE: 826.3086531128
MAE: 1020.2049559845187
MAE: 4142.808312700589
MAE: 6810.86558414305
MAE: 2802.18996643312
MAE: 801.2555454398142
MAE: 843.6299288432291
MAE: 3121.1144941877783
MAE: 4430.90615396231
MAE: 447.98300792477005
MAE: 7198.671915697901
MAE: 2417.581092625428
MAE: 530.4190047220792
MAE: 3236.844608024017
MAE: 986.8987966238893
MAE: 6656.554322996503
MAE: 340.7136809595199
MAE: 893.2874045985072
MA

In [37]:
# Obtener el mejor modelo con los mejores hiperparámetros
xgb_params_best = {
    'learning_rate': best_params['learning_rate'],
    'n_estimators': best_params['n_estimators'],
    'max_depth': best_params['max_depth'],
    'max_leaves': best_params['max_leaves'],
    'min_child_weight': best_params['min_child_weight'],
    'reg_alpha': best_params['reg_alpha'],
    'reg_lambda': best_params['reg_lambda'],
}

Nuevo_one_hot_parametro = one_hot(min_frequency_param=best_params['min_frequency'])
separacion_func = FunctionTransformer(separacion, validate=False)

#Implementacion ColumnTransformer
ct = ColumnTransformer(transformers = [
('Procesamiento_numericas',StandardScaler(),numeric_var),
('Procesamiento_categoricas', Nuevo_one_hot_parametro,categ_var)],remainder="passthrough",verbose_feature_names_out=False)

ct.set_output(transform='pandas')

#Generacion pipeline
best_model = Pipeline([
('function_transform',separacion_func),
('transformacion',ct),
('regresor', xgb.XGBRegressor(**xgb_params_best))
])


# Entrenar el mejor modelo con todos los datos
best_model.fit(X_train_final, Y_train_final)
val_predictions_xgb_opt = best_model.predict(X_val_final)
mae_xgb_opt = mean_absolute_error(Y_val_final, val_predictions_xgb_opt)
print(f'MAE con XGBoost Optimizado: {mae_xgb_opt}')
print(f'Número de trials: {num_trials}')
#Mostramos los hiperparámetros del mejor modelo
print('')
print('Mejores hiperparámetros:')
#Imprimimos los hiperparámetros del modelo del diccionario best_params de la forma: hiperparámetro: valor
for key, value in best_params.items():
    print(f'{key}: {value}')

# Guardar el modelo en un archivo .pkl
joblib.dump(best_model, 'best_model_optimized_con_prune.pkl')

MAE con XGBoost Optimizado: 2094.7839817223653
Número de trials: 125

Mejores hiperparámetros:
learning_rate: 0.07722139907687305
n_estimators: 978
max_depth: 8
max_leaves: 99
min_child_weight: 2
reg_alpha: 0.2385838098063926
reg_lambda: 0.23342325029414504
min_frequency: 0.029518033187261532


['best_model_optimized_con_prune.pkl']

¿Qué es prunning? ¿De qué forma debería impactar en el entrenamiento?

El prunning o podado, corresponde al proceso en donde se eliminan nodos asociados a los arboles que no aportan al desempeño del modelo durante el proceso de entrenamiento. Esto ocurre debido a que en varios casos el numero de nodos presentes en el arbol es demasiado grande lo que genera que existan nodos no relevantes que incluso pueden llegar a generar overfitting, debidoa esto, el prunning busca dichos nodos y los elimina con tal de no considerar sus pesos para la clasificacion. Esto impacta al entrenamiento debido a que agrega procesos de busqueda y eliminacion de dichos nodos, lo cual altera el desempeño final del modelo. Cabe destacar que aplicar prunning no necesarimanete mejora el MAE o Accuracy, debido a que en muchos casos al final del entrenamiento el resultado mostrado ya esta overfitieado, lo que implica que se presente un MAE muy pequeño. Esto se observa en los dos modelos optimizados, en donde al final del entrenamiento existe un MAE de 174.2933817167441 lo cual es muy bajo, pero dicho resultado no refleja un desempeño real dado que esta calculado con el conjunto con el que se entreno, basicamente esta overfiteado para dicho conjunto.

¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto?

Para este caso se obtuvieron los mismos resultados, esto se puede deber al parametro max_leaves el cual tiene un rango de (0,100), en donde si obervamos el resultado final de ambos casos se obtiene que el corresponde a 99, siendo el valor maximo que puede alcanzar. Por esto ultimo, se puede inferir que el numero de leaves no es suficiente para el algoritmo y en cosecuencia no deben existir muchas hojas que no influyan al desempeño del modelo y por ello no debe haber mucho podado, generando el mismo resultado para ambos metodos de optimizacion. Ademas cabe destacar que el proceso de optimizacion es iterativo, en donde se busca minimizar el MAE en cada iteracion, por ello, si el numero de hojas fuera muy grande teniendo muchos nodos innecesarios probablmente el MAE sea grande, a lo cual el algoritmo buscaria de forma automatica reducir dicho numero sin necesariamente aplicar prunning, en cambio, si estuvieramos trabajando con parametros fijos se verian cambios directos en los resultados finales.

## 1.5 Visualizaciones (0.5 puntos)

<p align="center">
  <img src="https://media.tenor.com/F-LgB1xTebEAAAAd/look-at-this-graph-nickelback.gif">
</p>


Satisfecho con su trabajo, Fiu le pregunta si es posible generar visualizaciones que permitan entender el entrenamiento de su modelo.

A partir del siguiente <a href = https://optuna.readthedocs.io/en/stable/tutorial/10_key_features/005_visualization.html#visualization>enlace</a>, genere las siguientes visualizaciones:

- Gráfico de historial de optimización
- Gráfico de coordenadas paralelas
- Gráfico de importancia de hiperparámetros

Comente sus resultados: ¿Desde qué *trial* se empiezan a observar mejoras notables en sus resultados? ¿Qué tendencias puede observar a partir del gráfico de coordenadas paralelas? ¿Cuáles son los hiperparámetros con mayor importancia para la optimización de su modelo?

In [38]:
# Inserte su código acá
from optuna.visualization import plot_optimization_history, plot_parallel_coordinate, plot_param_importances

plot_optimization_history(study)

¿Desde qué *trial* se empiezan a observar mejoras notables en sus resultados?

Desde la trial 17 aproximadamente, en donde se redujo el MAE de 10k hasta 1k, posterior a esto se reduce en gran medida para el trial 37 aprox. Pasado dichos valores ya se esta en escala de centesimas.

In [39]:
plot_parallel_coordinate(study)

¿Qué tendencias puede observar a partir del gráfico de coordenadas paralelas?

Se pueden observar las siguientes tendencias:

1) Aumentar el lerning rate se traduce en un aumento del MAE (peor desempeño)
2) Disminuir la min_frquency genera una disminucion del MAE (mejor desempeño)
3) Aumenar n_estimators aumenta el MAE (pero desempeño)
4) disminuir el reg_alpha disminuye el MAE (Este directamente no implica mejor o pero desempeño, dado que tiene relacion con el overfitting).

Dichas tendencias son las mas claras.

In [40]:
plot_param_importances(study)

¿Cuáles son los hiperparámetros con mayor importancia para la optimización de su modelo?

El parametro con mayor importancia corresponde al min_frequency, esto tiene sentido dada la explicacion de la seccion anterior, en donde alterar el valor de dicho parametro se traduce en codificaciones distintas de las variables categoricas lo que genera distintos resultados en el modelo. Basicamente es un paraemtro muy sensible dado que altera la configuracion de la data, priorizando ciertas categorias en comparacion a otras.
La segunda corresponde a n_estimators referida al numero de arboles, esta tambien tiene bastante logica debido a que un mayor numero implica una mayor complejidad del modelo, lo cual tambien se asocia con max_depth. Nuevamente estos parametros tienen mas importancia en relacion a la complicacion del modelo, en donde no tiene sentido generar un modelo con muchas capas si el problema de clasificacion es simple.

Por ultimo esta reg_alpha, dado que tiene mas importancia que el resto probablemente se traduce en que existe un cierto porcentaje de overfitting en el modelo. Aun asi, es baja por lo cual no necesariamente es un gran problema.

## 1.6 Síntesis de resultados (0.3)

Finalmente, genere una tabla resumen del MAE obtenido en los 5 modelos entrenados (desde Baseline hasta XGBoost con Constraints, Optuna y Prunning) y compare sus resultados. ¿Qué modelo obtiene el mejor rendimiento?

Modelos:
- Dummy = 13546.494391911923
- XGB normal = 2390.122370125702
- XGB Opti = 2094.7839817223653
- XGB Opti prunning = 2094.7839817223653

¿Qué modelo obtiene el mejor rendimiento?
Los dos modelos que obtuvieron el mejor rendimiento, fueron los dos optimizados, teniendo el mismo resultado de 2094.7839817223653.

Por último, cargue el mejor modelo, prediga sobre el conjunto de test y reporte su MAE. ¿Existen diferencias con respecto a las métricas obtenidas en el conjunto de validación? ¿Porqué puede ocurrir esto?

In [41]:
# Obtener el mejor modelo con los mejores hiperparámetros (los ultimos dos)
xgb_params_best = {
    'learning_rate': 0.07722139907687305,
    'n_estimators': 978,
    'max_depth': 8,
    'max_leaves': 99,
    'min_child_weight': 2,
    'reg_alpha': 0.2385838098063926,
    'reg_lambda': 0.23342325029414504,
}

best_min_frequency = 0.029518033187261532

Nuevo_one_hot_parametro = one_hot(min_frequency_param=best_min_frequency)
separacion_func = FunctionTransformer(separacion, validate=False)

#Implementacion ColumnTransformer
ct = ColumnTransformer(transformers = [
('Procesamiento_numericas',StandardScaler(),numeric_var),
('Procesamiento_categoricas', Nuevo_one_hot_parametro,categ_var)],remainder="passthrough",verbose_feature_names_out=False)

ct.set_output(transform='pandas')

#Generacion pipeline
best_model = Pipeline([
('function_transform',separacion_func),
('transformacion',ct),
('regresor', xgb.XGBRegressor(**xgb_params_best))
])


# Entrenar el mejor modelo con todos los datos
best_model.fit(X_test_final, Y_test_final)

val_predictions_xgb_opt = best_model.predict(X_test_final)
mae_xgb_opt = mean_absolute_error(Y_test_final, val_predictions_xgb_opt)
print(f'MAE con XGBoost Optimizado: {mae_xgb_opt}')
print(f'Número de trials: {num_trials}')
#Mostramos los hiperparámetros del mejor modelo
print('')
print('Mejores hiperparámetros:')
#Imprimimos los hiperparámetros del modelo del diccionario best_params de la forma: hiperparámetro: valor
for key, value in best_params.items():
    print(f'{key}: {value}')

# Guardar el modelo en un archivo .pkl
joblib.dump(best_model, 'best_model_optimized_final.pkl')

MAE con XGBoost Optimizado: 0.4524832308771783
Número de trials: 125

Mejores hiperparámetros:
learning_rate: 0.07722139907687305
n_estimators: 978
max_depth: 8
max_leaves: 99
min_child_weight: 2
reg_alpha: 0.2385838098063926
reg_lambda: 0.23342325029414504
min_frequency: 0.029518033187261532


['best_model_optimized_final.pkl']

¿Existen diferencias con respecto a las métricas obtenidas en el conjunto de validación? ¿Porqué puede ocurrir esto?

Se obtuvo un MAE de 0.4524832308771783 lo cual resulta en un muy buen resultado. Si existen diferencias respecto a las metricas del conjunto de validacion, esto ocurre por que en esencia son distintos datos, teniendo comportamientos distintos y generando valores de metricas diferentes.

En general, siempre se obtienen valores distintos pero en este caso se obtuvo una diferencia muy grande, esto es fuera de lo comun siendo coincidencia para este caso. Cabe destacar que el 10% de los datos corresponde a 746 muestras, siendo un valor pequeño, como el MAE es muy bajo esto se traduce en que para casi todas esas muestras las predicciones fueron correctas lo cual no necesariamente implica que el clasificador predice siempre de forma correcta, sino mas bien que para esos 746 ejemplos si lo hizo.

Cabe destacar que la funcion train_test_split separa los conjuntos manteniendo la representatividad de las categorias, lo que garantiza que los conjunots sean representativos por ende no deberia existir un error asociado a la variabilidad. Aun asi parece sospechoso el resultado y se destaca nuevamente la pqueñez del conjunto probado.

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por mail o U-cursos.

<p align="center">
  <img src="https://media.tenor.com/8CT1AXElF_cAAAAC/gojo-satoru.gif">
</p>

#Cuando vemos los resultados del XGBOOST

<p align="center">
  <img src="https://tenor.com/es-419/view/sukuna-you're-strong-sukuna-jogo-strong-sukuna-be-proud-sukuna-won-gif-18267921484872102678.gif">
</p>

#XGB OPTIMIZADO

<p align="center">
  <img src="https://tenor.com/es-419/view/jojo-bizzare-adventure-kars-jojo-gif-25698437.gif">
</p>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>